This package extends the openai and langchain packages to support consumption of the AI Core LLM service (Azure OpenAI).

Usage is as follows.

# OpenAI-like API

## LLM's

`BTPCompletion` and `BTPChatCompletion` equivalent to `openai.Completion` and `openai.ChatCompletion`.

In [1]:
from llm_commons.btp_llm.openai import BTPCompletion
response = BTPCompletion.create(
  deployment_id="text-davinci-003",
  prompt="Say this is a test",
  max_tokens=7,
  temperature=0
)
print(response)


{
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": "\n\nThis is indeed a test"
    }
  ],
  "created": 1689798230,
  "id": "cmpl-7e806aIrP2quW8Qcb0zCgELivZPt8",
  "model": "text-davinci-003",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 7,
    "prompt_tokens": 5,
    "total_tokens": 12
  }
}


In [2]:
from llm_commons.btp_llm.openai import BTPChatCompletion

messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Does Azure OpenAI support customer managed keys?"},
        {"role": "assistant", "content": "Yes, customer managed keys are supported by Azure OpenAI."},
        {"role": "user", "content": "Do other Azure Cognitive Services support this too?"}
    ]

response = BTPChatCompletion.create(
    deployment_id="gpt-4", # engine = "deployment_name".
    messages=messages
)
print(response)

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Yes, many Azure Cognitive Services support customer-managed keys as part of Azure's commitment to providing robust security for its users. To get the specifics for each service, you should check the individual documentation for each Azure Cognitive Service.",
        "role": "assistant"
      }
    }
  ],
  "created": 1689798231,
  "id": "chatcmpl-7e807k8v8CTTTTGtJiefwAOBUm2a4",
  "model": "gpt-4",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 45,
    "prompt_tokens": 55,
    "total_tokens": 100
  }
}


## Embeddings

`BTPEmbedding` is equivalent to `openai.Embedding`.

In [3]:
from llm_commons.btp_llm.openai import BTPEmbedding

response = BTPEmbedding.create(
    input="Your text string goes here",
    deployment_id="text-embedding-ada-002-v2"
)
response['data'][0]['embedding']

[-0.006929283495992422,
 -0.005336422007530928,
 0.01187589205801487,
 -0.025001877918839455,
 -0.02469271421432495,
 0.039787933230400085,
 -0.010101565159857273,
 -0.00940258800983429,
 -0.013186474330723286,
 -0.009940262883901596,
 -0.011680984869599342,
 0.007870214991271496,
 -0.014087079092860222,
 0.007769400719553232,
 0.010168774053454399,
 -0.005033980123698711,
 0.02294527180492878,
 -0.001512210350483656,
 0.014933916740119457,
 -0.01026286743581295,
 0.004862596280872822,
 0.012460612691938877,
 0.004909642972052097,
 0.010807262733578682,
 -0.006603318266570568,
 -0.00035746971843764186,
 0.0055783758871257305,
 -0.012494217604398727,
 0.0163721963763237,
 0.004529910162091255,
 0.0065629929304122925,
 -0.007070423103868961,
 -0.015216195955872536,
 -0.006589876487851143,
 -0.01872452348470688,
 0.0040796073153615,
 0.003212606767192483,
 -0.018993360921740532,
 0.030163554474711418,
 -0.007594656199216843,
 0.008233144879341125,
 0.00942275021225214,
 -0.001088791410438

# `langchain` Integration

# LLM

In [4]:
from llm_commons.langchain.btp_llm import BTPOpenAI  # langchain class representing the BTP OpenAI models
from langchain import PromptTemplate, LLMChain

deployment_id = "text-davinci-003"

llm = BTPOpenAI(deployment_id=deployment_id)  # can be used as usual with langchain

template = """Question: {question}

Answer: Let's think step by step."""
prompt = PromptTemplate(template=template, input_variables=["question"])
llm_chain = LLMChain(prompt=prompt, llm=llm, verbose=True)

question = "What NFL team won the Super Bowl in the year Justin Beiber was born?"

print(llm_chain.predict(question=question))




> Entering new  chain...
Prompt after formatting:
Question: What NFL team won the Super Bowl in the year Justin Beiber was born?

Answer: Let's think step by step.

> Finished chain.
 Justin Bieber was born in 1994, so the NFL team that won the Super Bowl in 1994 was the Dallas Cowboys.


## Chat Model

In [5]:
from llm_commons.langchain.btp_llm import ChatBTPOpenAI  # langchain class representing the BTP OpenAI models

deployment_id = "gpt-4"

chat_llm = ChatBTPOpenAI(deployment_id)  # can be used as usual with langchain

In [6]:
from langchain import LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

template = "You are a helpful assistant that translates english to pirate."

system_message_prompt = SystemMessagePromptTemplate.from_template(template)

example_human = HumanMessagePromptTemplate.from_template("Hi")
example_ai = AIMessagePromptTemplate.from_template("Ahoy!")
human_template = "{text}"

human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages(
    [system_message_prompt, example_human, example_ai, human_message_prompt])

chain = LLMChain(llm=chat_llm, prompt=chat_prompt)  # pass the BTP OpenAI model

# get a chat completion from the formatted messages
print(chain.run("I love programming"))

I be fond o' codin'.


## Embedding


In [7]:
from llm_commons.langchain.btp_llm import BTPOpenAIEmbeddings

embeddings = BTPOpenAIEmbeddings(
    deployment_id="text-embedding-ada-002-v2"
)
text = "This is a test query."

embeddings.embed_query(text)

[-0.005052816588431597,
 0.005106533877551556,
 -0.005156894214451313,
 -0.019163770601153374,
 -0.025542741641402245,
 0.02274942398071289,
 -0.014288894832134247,
 -0.012066327035427094,
 -0.02727513574063778,
 -0.035185061395168304,
 0.01052865944802761,
 0.013288403861224651,
 0.009293152950704098,
 -0.0004081282240804285,
 0.0022645345889031887,
 0.003352317027747631,
 0.0207484420388937,
 -0.0037098752800375223,
 0.02332688868045807,
 -0.013939729891717434,
 0.000148877632454969,
 -0.0041966913267970085,
 0.0035487222485244274,
 0.016585323959589005,
 -0.014127741567790508,
 -0.0056940708309412,
 0.01017277967184782,
 -0.027288565412163734,
 0.003303635399788618,
 -0.00203119870275259,
 0.010683097876608372,
 0.001400855602696538,
 -0.009373730048537254,
 -0.024884700775146484,
 -0.016236159950494766,
 0.005089747253805399,
 0.0057578603737056255,
 -0.005143465008586645,
 0.025032425299286842,
 -0.00038105956627987325,
 0.017646247521042824,
 0.008930559270083904,
 -0.00270770536

**QA with embeddings**

This example requires chromadb to be installed. Install via `export HNSWLIB_NO_NATIVE=1 && pip install chromadb`.

For a dedicated project on QA with embeddings, visit https://github.wdf.sap.corp/AI/qa-with-embeddings.


In [10]:
from langchain.chains import RetrievalQA

from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.vectorstores import Chroma

loader = TextLoader('./state_of_the_union.txt')
documents = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

EMBEDDING_MODEL_LOCAL = "text-embedding-ada-002-v2"
embeddings = BTPOpenAIEmbeddings(deployment_id=EMBEDDING_MODEL_LOCAL, chunk_size=1)

db = Chroma.from_documents(texts, embeddings)

retriever = db.as_retriever()

qa = RetrievalQA.from_llm(
    llm=llm, retriever=retriever)  # pass the BTP OpenAI model

query = "What did the president say about Ketanji Brown Jackson"
print(qa.run(query))

 The president said that Ketanji Brown Jackson is one of the nation's top legal minds and that she will continue Justice Breyer's legacy of excellence.
